In [1]:
# First we download the dataset of bach chorales, here this set is assumed to be the Inspiration Set
!wget -O Jsb16thSeparated.npz https://github.com/czhuang/JSB-Chorales-dataset/raw/master/Jsb16thSeparated.npz

--2020-05-16 16:20:25--  https://github.com/czhuang/JSB-Chorales-dataset/raw/master/Jsb16thSeparated.npz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/czhuang/JSB-Chorales-dataset/master/Jsb16thSeparated.npz [following]
--2020-05-16 16:20:26--  https://raw.githubusercontent.com/czhuang/JSB-Chorales-dataset/master/Jsb16thSeparated.npz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102808 (100K) [application/octet-stream]
Saving to: ‘Jsb16thSeparated.npz’

Jsb16thSeparated.np 100%[===================>] 100.40K   375KB/s    in 0.3s    

2020-05-16 16:20:26 (375 KB/s) - ‘Jsb16thSeparated.npz’ saved [102808/102808]



In [0]:
# We import required packages
import numpy as np
from magenta.models.coconet import lib_pianoroll
from magenta.models.coconet import lib_util
import pretty_midi
import os

In [0]:
# We now read the file contents
p = open("Jsb16thSeparated.npz", "rb")
dataset_bach_chorales = np.load(p, allow_pickle=True, encoding="latin1")

In [4]:
dataset_bach_chorales['valid'].shape

(76,)

In [5]:
dataset_bach_chorales['train'].shape

(229,)

In [6]:
dataset_bach_chorales['test'].shape

(77,)

In [0]:
# We define Hiperparameters
key = "Jsb16thSeparated"
min_pitch = 36
max_pitch = 81
shortest_duration = 0.125
quantization_level = 0.125
num_instruments = 4
qpm = 60
separate_instruments = True

In [0]:
# We define the pianoroll encoder_decoder
pianoroll_encoder_decoder = lib_pianoroll.PianorollEncoderDecoder(
      shortest_duration=shortest_duration,
      min_pitch=min_pitch,
      max_pitch=max_pitch,
      separate_instruments=separate_instruments,
      num_instruments=num_instruments,
      quantization_level=quantization_level)

In [0]:
# We get the piano rolls
train_pianorolls = list(map(pianoroll_encoder_decoder.encode, dataset_bach_chorales['train']))

In [0]:
# From the pianorolls we get the midi format
midi_datas = []
for pianoroll in train_pianorolls:
  midi_data = pianoroll_encoder_decoder.decode_to_midi(pianoroll)
  midi_datas.append(midi_data)

In [11]:
# We save the midis
for i, midi_data in enumerate(midi_datas):
    midi_fpath = os.path.join("./TrainMidis", "%s_%i.midi" % ("TrainMidi", i))
    with lib_util.atomic_file(midi_fpath) as p:
      midi_data.write(p)

In [0]:
# We obtain the k elements for the inspiration set
inspiration_bag = {}
inspiration_set = {}
for pianoroll in train_pianorolls:
  pianoroll_bag = []
  for timestep in pianoroll:
    timestep_bag = []
    for pitchstep in timestep:
      pitches_list = pitchstep.tolist()
      timestep_bag.append("".join(map(str, map(int,pitches_list))))
    timestep_string = "".join(timestep_bag)
    pianoroll_bag.append(timestep_string)
    if timestep_string in inspiration_bag: 
      inspiration_bag[timestep_string] += 1 
    else: 
      inspiration_bag[timestep_string] = 1
  pianoroll_string = "".join(pianoroll_bag)
  if pianoroll_string in inspiration_set: 
    inspiration_set[pianoroll_string] += 1 
  else: 
    inspiration_set[pianoroll_string] = 1

In [0]:
# We obtain the frequencies
repeated_values = []
for key in inspiration_set:
  repeated_values.append(inspiration_set[key])
repeated_values = np.array(repeated_values)

In [0]:
# Read generated Midis
generated_midis_path = "GeneratedMidis/"
label_midis = "GeneratedMidi_"
generated_pianorolls = []
for i in range(50):
  current_generated_midi_path =  generated_midis_path + label_midis + str(i) + ".midi"
  current_generated_midi_data = pretty_midi.PrettyMIDI(current_generated_midi_path)
  current_generated_pianoroll = pianoroll_encoder_decoder.encode_midi_to_pianoroll(current_generated_midi_data,(1,32,46,4))
  generated_pianorolls.append(current_generated_pianoroll[0])

In [0]:
bags_R = {}
for generated_pianoroll in generated_pianorolls:
  pianoroll_bag = []
  
  for timestep in generated_pianoroll:
    timestep_bag = []
    for pitchstep in timestep:
      pitches_list = pitchstep.tolist()
      timestep_bag.append("".join(map(str, map(int,pitches_list))))
    timestep_string = "".join(timestep_bag)
    pianoroll_bag.append(timestep_string)

  pianoroll_string = "".join(pianoroll_bag)
  is_outside_inspiration_set = True
  
  if pianoroll_string in inspiration_set: 
    is_outside_inspiration_set = False

  if (is_outside_inspiration_set):
    for timestep_string in pianoroll_bag:
      if timestep_string in bags_R: 
        bags_R[timestep_string] += 1 
      else: 
        bags_R[timestep_string] = 1

In [64]:
af_bags_R = 0
for k in bags_R:
    af_bags_R += bags_R[k]
af_bags_R = af_bags_R / len(bags_R)
af_bags_R

3.10077519379845

In [0]:
rf_per_k = {}
for k in bags_R:
    rf_per_k[k] = bags_R[k] / af_bags_R

In [0]:
complexities_ii2 = []
for generated_pianoroll in generated_pianorolls:
  pianoroll_bag = []
  
  for timestep in generated_pianoroll:
    timestep_bag = []
    for pitchstep in timestep:
      pitches_list = pitchstep.tolist()
      timestep_bag.append("".join(map(str, map(int,pitches_list))))
    timestep_string = "".join(timestep_bag)
    pianoroll_bag.append(timestep_string)

  pianoroll_string = "".join(pianoroll_bag)
  is_outside_inspiration_set = True
  
  if pianoroll_string in inspiration_set: 
    is_outside_inspiration_set = False

  if (is_outside_inspiration_set):
    complexity_ii2 = 0
    for element_k in pianoroll_bag:
      complexity_ii2 += rf_per_k[element_k]
    complexities_ii2.append(1.0 / complexity_ii2)
complexities_ii2 = np.array(complexities_ii2)

In [89]:
np.median(complexities_ii2),np.mean(complexities_ii2)

(0.017226528854435826, 0.01751810701233277)

In [98]:
np.sort(complexities_ii2 * 100)

array([0.34723126, 0.37539651, 0.42828387, 0.4969191 , 0.50666261,
       0.53277924, 0.56583489, 0.69524107, 0.73652617, 0.75628663,
       0.77908924, 0.88090204, 0.95115803, 0.96297366, 1.05468544,
       1.07665805, 1.10741971, 1.14843526, 1.15700567, 1.18350198,
       1.21599027, 1.4907573 , 1.4907573 , 1.53503722, 1.72265289,
       1.72265289, 1.74200854, 1.82398541, 1.82398541, 1.86793686,
       1.86793686, 1.89071658, 1.91405876, 1.98767641, 2.06718346,
       2.12381863, 2.50062516, 2.67308206, 2.76854928, 2.76854928,
       2.81888654, 2.87108814, 2.92525962, 2.98151461, 2.98151461,
       3.03997568, 3.10077519, 3.26397389, 4.30663221, 4.55996352])